<a href="https://colab.research.google.com/github/zal-eun/khuda_recsys_hm/blob/main/als.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#articles = pd.read_csv("/content/drive/MyDrive/Khuda3차/초기자료/articles.csv")
#customers = pd.read_csv("/content/drive/MyDrive/Khuda3차/초기자료/customers.csv")
transactions = pd.read_csv("/content/drive/MyDrive/Khuda3차/초기자료/transactions_train.csv")

In [ ]:
transactions= transactions[['customer_id', 'article_id']]

In [ ]:
# '0719530003' 값을 가지는 행만 추출하여 새로운 DataFrame으로 가져오기
selected_rows = transactions[transactions['customer_id'] == '0000757967448a6cb83efb3ea7a3fb9d418ac7adf2379d8cd0c725276a467a2a']

# 결과 확인
print(selected_rows)






                                                customer_id  article_id
11324052  0000757967448a6cb83efb3ea7a3fb9d418ac7adf2379d...   735843004
11324053  0000757967448a6cb83efb3ea7a3fb9d418ac7adf2379d...   726925001
18148893  0000757967448a6cb83efb3ea7a3fb9d418ac7adf2379d...   715624008
18148894  0000757967448a6cb83efb3ea7a3fb9d418ac7adf2379d...   783388001
31492019  0000757967448a6cb83efb3ea7a3fb9d418ac7adf2379d...   719530003
31492020  0000757967448a6cb83efb3ea7a3fb9d418ac7adf2379d...   448509014


In [ ]:
!pip install pyspark -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import pyspark
from pyspark.sql import SparkSession

import warnings
warnings.filterwarnings("ignore")

spark = SparkSession.builder.appName("fashion-recommendations").getOrCreate()

In [ ]:
transactions = (spark.read.format("csv")
                .option("header", "true")
                .load("/content/drive/MyDrive/Khuda3차/초기자료/transactions_train.csv"))
transactions.printSchema()


root
 |-- t_dat: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- article_id: string (nullable = true)
 |-- price: string (nullable = true)
 |-- sales_channel_id: string (nullable = true)



In [ ]:
from pyspark.sql.functions import *

transactions = transactions.withColumn('t_dat', to_date("t_dat"))

In [ ]:
# filter only 2020
sales =  transactions.withColumn('t_dat', transactions['t_dat'].cast('string'))
sales = sales.withColumn('date', from_unixtime(unix_timestamp('t_dat', 'yyyy-MM-dd')))
sales = sales.withColumn('year', year(col('date')))
sales = sales.withColumn('month', month(col('date')))

# Let's filter the data to start with
sales = sales[sales['year'] == 2020]
sales = sales[sales['month'] == 1]

transactions.unpersist()

# Prepare the dataset
sales = sales.groupby('customer_id', 'article_id').count()
sales.show(5)

+--------------------+----------+-----+
|         customer_id|article_id|count|
+--------------------+----------+-----+
|0101b86c9ea4581da...|0822138002|    1|
|05b6661e851f78bc0...|0783517002|    1|
|16d13b0db3658a9fd...|0801327001|    1|
|17f52fdadc1209858...|0762846008|    1|
|1de87318baafea4da...|0788633001|    1|
+--------------------+----------+-----+
only showing top 5 rows



In [ ]:

from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

customer id를 인데스로 변환


In [ ]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.sql.functions import col

indexer = [StringIndexer(inputCol=column, outputCol=column+"_index") for column in list(set(sales.columns)-set(['count'])) ]
pipeline = Pipeline(stages=indexer)
transformed = pipeline.fit(sales).transform(sales)
transformed.show(5)

+--------------------+----------+-----+----------------+-----------------+
|         customer_id|article_id|count|article_id_index|customer_id_index|
+--------------------+----------+-----+----------------+-----------------+
|0101b86c9ea4581da...|0822138002|    1|         10419.0|          14556.0|
|05b6661e851f78bc0...|0783517002|    1|          1739.0|          39354.0|
|16d13b0db3658a9fd...|0801327001|    1|          3228.0|           1107.0|
|17f52fdadc1209858...|0762846008|    1|            61.0|          70733.0|
|1de87318baafea4da...|0788633001|    1|          3222.0|           1274.0|
+--------------------+----------+-----+----------------+-----------------+
only showing top 5 rows



사용자:구매항목 데이터셋 8:2로 분리


In [ ]:
(training,test)=transformed.randomSplit([0.8, 0.2])

als=ALS(maxIter=5,regParam=0.09,rank=25,userCol="customer_id_index",itemCol="article_id_index",ratingCol="count",coldStartStrategy="drop",nonnegative=True)
model=als.fit(training)

In [ ]:
evaluator=RegressionEvaluator(metricName="rmse",labelCol="count",predictionCol="prediction")
predictions=model.transform(test)
rmse=evaluator.evaluate(predictions)
print("RMSE="+str(rmse))
predictions.show(5)

RMSE=0.5255790318061214
+--------------------+----------+-----+----------------+-----------------+----------+
|         customer_id|article_id|count|article_id_index|customer_id_index|prediction|
+--------------------+----------+-----+----------------+-----------------+----------+
|da624600418470043...|0428291016|    1|           917.0|              1.0| 0.9377146|
|da624600418470043...|0428291017|    1|          3963.0|              1.0|0.90688604|
|da624600418470043...|0752814006|    1|          1428.0|              1.0| 0.9666817|
|da624600418470043...|0754362006|    1|           705.0|              1.0| 0.9435724|
|da624600418470043...|0764520002|    1|          1346.0|              1.0| 0.9159045|
+--------------------+----------+-----+----------------+-----------------+----------+
only showing top 5 rows



In [ ]:
# Assuming you have a dictionary article_id_to_image_file that maps article_id_index to image file paths

def load_image_for_customer(customer_id):
    # Filter predictions to get rows corresponding to the specified customer_id
    customer_predictions = predictions.filter(predictions['customer_id_index'] == customer_id)

    # Get the first row of the customer's predictions (you can modify this depending on your requirements)
    prediction_row = customer_predictions.head(1)[0]

    # Get the corresponding article_id_index from the prediction row
    article_id_index = int(prediction_row['article_id_index'])

    # Assuming article_id_to_image_file is a dictionary mapping article_id_index to image file paths
    image_file_path = article_id_to_image_file.get(article_id_index)

    if image_file_path:
        # Here you can add code to load the image using the image_file_path
        # For example, using a library like PIL (Python Imaging Library)
        from PIL import Image
        image = Image.open(image_file_path)
        image.show()
    else:
        print("Image not found for the specified customer and article.")


In [ ]:
# Assuming you have a dictionary article_id_to_image_file that maps article_id_index to image file paths

def load_image_for_customer(customer_id):
    # ... (same as before)

# Get the first customer_id_index from the predictions DataFrame
  first_customer_id_index = predictions.select("customer_id_index").head(1)[0][0]

# Call the load_image_for_customer function with the first_customer_id_index
  load_image_for_customer(first_customer_id_index)
